# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [40]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,  StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score, mean_squared_error


%matplotlib inline

In [41]:
df = pd.read_csv('previsao_de_renda.csv')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [43]:
# Removendo a coluna 'Unnamed: 0'
df = df.drop('Unnamed: 0', axis=1)

# Exibindo o DataFrame resultante
print(df)

         data_ref  id_cliente sexo  posse_de_veiculo  posse_de_imovel  \
0      2015-01-01       15056    F             False             True   
1      2015-01-01        9968    M              True             True   
2      2015-01-01        4312    F              True             True   
3      2015-01-01       10639    F             False             True   
4      2015-01-01        7064    M              True            False   
...           ...         ...  ...               ...              ...   
14995  2016-03-01       16006    F             False             True   
14996  2016-03-01        3722    F             False             True   
14997  2016-03-01        6194    F              True             True   
14998  2016-03-01        4922    M              True            False   
14999  2016-03-01       14651    F             False             True   

       qtd_filhos        tipo_renda           educacao estado_civil  \
0               0        Empresário         Secundár

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [44]:
# Separando as variáveis independentes (X) da variável dependente (y)
X = df.drop(['renda'], axis=1)
y = np.log1p(df['renda'])  # Usar log1p para lidar com valores zero

# Dividindo o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definindo as transformações para variáveis numéricas e categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = X.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinando as transformações
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Criando o modelo de regressão linear múltipla
modelo_regressao_linear = LinearRegression()

# Criando o pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', modelo_regressao_linear)
])

# Treinando o modelo usando o pipeline
pipeline.fit(X_train, y_train)

# Avaliando o desempenho do modelo
score = pipeline.score(X_test, y_test)
print(f'R² do modelo: {score}')

# Fazendo previsões usando o modelo treinado
y_pred = pipeline.predict(X_test)

R² do modelo: 0.35014473255479817


In [50]:
def preprocess_data(df):    
    if 'data_ref' in df.columns:
        # Excluir coluna 'data_ref'
        df = df.drop('data_ref', axis=1)

    # Tratando valores ausentes
    imputer = SimpleImputer(strategy='mean')
    df['renda'] = imputer.fit_transform(df[['renda']])

    # Tratando valores infinitos
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    # Convertendo variáveis categóricas 
    df = pd.get_dummies(df, drop_first=True)

    return df

df = preprocess_data(df)

# Imprimindo informações sobre os tipos de dados
print(df.dtypes)

df['renda'] = pd.to_numeric(df['renda'], errors='coerce')

X = df.drop('renda', axis=1)  
y = df['renda']  


# Aplicando a eliminação backward
X_optimized = backward_elimination(X, y)

X_optimized = backward_elimination(X, y)

# Exibindo informações sobre os tipos de dados
print(X_optimized.dtypes)

# Convertendo todas as colunas para float
X_optimized = X_optimized.astype(float)

# Treinando o modelo com as variáveis otimizadas
X_optimized_with_intercept = sm.add_constant(X_optimized)
model_after_elimination = sm.OLS(y, X_optimized_with_intercept).fit()
print(model_after_elimination.summary())

id_cliente                         int64
posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_M                              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_Casa                bool
tipo_residencia_Com os pais         bool
tipo_residencia_Comunitário         bool
tipo_residencia_

In [49]:
def backward_elimination(X, y, significance_level=0.05):
    # Adicionando uma coluna de constante ao conjunto de recursos
    X_with_intercept = sm.add_constant(X)
    
    while True:
        # Ajustando o modelo usando OLS 
        model = sm.OLS(y.astype(float), X_with_intercept.astype(float)).fit()
        
        # Identificando a variável menos significativa
        least_significant_variable = model.pvalues[1:].idxmax()
        max_p_value = model.pvalues[1:].max()        
       
        if max_p_value > significance_level:
            X_with_intercept = X_with_intercept.drop(least_significant_variable, axis=1)
        else:
            
            break
    
    # Retornando apenas as variáveis significativas
    selected_features = X_with_intercept.drop('const', axis=1)
    
    return selected_features

X_optimized = backward_elimination(X, y)

# Treinando o modelo com as variáveis otimizadas
X_optimized_with_intercept = sm.add_constant(X_optimized)
model_after_elimination = sm.OLS(y.astype(float), X_optimized_with_intercept.astype(float)).fit()

# Comparando os resultados do modelo final
print(model_after_elimination.summary())

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     609.1
Date:                Mon, 29 Jan 2024   Prob (F-statistic):               0.00
Time:                        18:06:58   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12419   BIC:                         2.577e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

A eliminação de variáveis menos significativas não parece ter impactado significativamente os resultados do modelo. Talvez, Se algum aspecto específico fosse  discutido, seria possível realizar melhorias mais efetivas.